In [1]:
import warnings
warnings.filterwarnings('ignore')

## Import Libraries

In [27]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from sklearn.model_selection import train_test_split

## Load Data

In [37]:
# path to image files
infected_path = r'C:\Users\MQ0\Downloads\cell_images\Parasitized'
uninfected_path = r'C:\Users\MQ0\Downloads\cell_images\Uninfected'

# variables to store data
data = []
labels = []

# read in every image as numpy array
label = 0
for x in [uninfected_path,infected_path]:  
    for im in os.listdir(x):
        if 'Thumbs.db' not in im:
            image = Image.open(x + '\\' + im).resize((50,50))
            data.append(np.asarray(image))
            labels.append(label)
    label += 1

In [45]:
# convert data to numpy array
data = np.array(data)/255
labels = np.array(labels)

# split train test
X_train, X_test, y_train, y_test = train_test_split(data, labels, shuffle = True)